In [30]:
#2021.06.25. FRI 
#Hankyeong

#00. 패키지 호출
import pandas as pd 
import numpy as np
import warnings
import re
from konlpy.tag import Okt
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

#00-1. warning message ignore
warnings.filterwarnings(action='ignore')


In [2]:
#15. 네이버 영화 리뷰 데이터로 한글 감성 분석하기. & CountVectorizer
#    데이터셋 전처리 시행하기.  
#(1) 데이터셋 불러오기
review_train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',sep='\t') 
review_test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',sep='\t') 

#(2) train 데이터셋 살펴보기. 
review_train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [3]:
#(4) train 데이터셋 결측치 데이터 전처리하기. 
#①데이터셋 결측 여부 확인하기. 
review_train.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [4]:
#②데이터셋 결측 데이터 삭제하기.
review_train = review_train.dropna(how='any')
review_train.shape

(149995, 3)

In [5]:
#(4) train 데이터셋 중복여부 데이터 전처리하기. 
#①데이터셋 중복여부 확인하기. 
review_train['document'].nunique()

146182

In [6]:
#②데이터셋 중복 데이터 삭제하기.
review_train.drop_duplicates(subset=['document'], inplace=True)
review_train.shape

(146182, 3)

In [7]:
#(5) train 데이터셋의 target 변수 분포 확인하기. 
review_train['label'].value_counts()

0    73342
1    72840
Name: label, dtype: int64

In [8]:
#(6) test 데이터셋 결측치 데이터 전처리하기. 
#①데이터셋 결측 여부 확인하기. 
review_test.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [9]:
#②데이터셋 결측 데이터 삭제하기.
review_test = review_test.dropna(how='any')
review_test.shape

(49997, 3)

In [10]:
#(7) test 데이터셋 중복여부 데이터 전처리하기. 
#①데이터셋 중복여부 확인하기. 
review_test['document'].nunique()

49157

In [11]:
#②데이터셋 중복 데이터 삭제하기.
review_test.drop_duplicates(subset=['document'], inplace=True)
review_test.shape

(49157, 3)

In [12]:
#(8) test 데이터셋의 target 변수 분포 확인하기. 
review_test['label'].value_counts()

1    24711
0    24446
Name: label, dtype: int64

In [13]:
#16. 텍스트 데이터 전처리하기. 
#(1) train 데이터셋에 한글이 아닌 글자는 모두 삭제하기. 
review_train['document'] = review_train['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',"")
review_train

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [14]:
#(2) train 데이터셋에 전처리후 공백 데이터를 결측치 처리하기. 
#ⓛ데이터셋 결측처리 및 확인하기. 
review_train['document'].replace('',np.nan, inplace=True)
review_train['document'].isnull().sum()

391

In [15]:
#②데이터셋 중복 데이터 삭제하기.
review_train = review_train.dropna(how='any')
review_train.shape

(145791, 3)

In [16]:
#(3) test 데이터셋에 한글이 아닌 글자는 모두 삭제하기. 
review_test['document'] = review_test['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',"")
review_test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [17]:
#(4) test 데이터셋에 전처리후 공백 데이터를 결측치 처리하기. 
#ⓛ데이터셋 결측처리 및 확인하기. 
review_test['document'].replace('',np.nan, inplace=True)
review_test['document'].isnull().sum()

162

In [18]:
#②데이터셋 중복 데이터 삭제하기.
review_test = review_test.dropna(how='any')
review_test.shape

(48995, 3)

In [19]:
#PLUS. 전처리 완료한 데이터 저장하기. 
review_train.to_csv('D://Python_Project/Hankyeong_DataAnalysis/Data/naver_movie_review_train.csv', sep='\t',index=False)
review_test.to_csv('D://Python_Project/Hankyeong_DataAnalysis/Data/naver_movie_review_test.csv', sep='\t', index=False)

In [20]:
#17. 한글 처리하기. 
#(1) 객체 설정하기. 
okt = Okt()

#(2) 불용어 설정하기. 
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

#(3) 명사추출과 불용어처리해 X_train 데이터셋 할당하기. 
X_train = []
for sentence in tqdm_notebook(review_train['document']) :
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_train.append(temp_X)

  0%|          | 0/145791 [00:00<?, ?it/s]

In [21]:
#(4) 명사추출과 불용어처리해 X_test 데이터셋 할당하기. 
X_test = [] 
for sentence in tqdm_notebook(review_test['document']) :
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_test.append(temp_X)

  0%|          | 0/48995 [00:00<?, ?it/s]

In [22]:
#PLUS. y_train, y_test 지정하기. 
y_train = review_train['label'].values
y_test = review_test['label'].values

#memo. values 어트리뷰트로 시리즈 -> 넘파이 어레이 변환함. 

In [23]:
#18. Count Vectorizer로 텍스트 데이터 벡터화 하기. 
#(1) Count Vectorizer 객체 생성하기. 
count_vect = CountVectorizer()

#(2) X_train 데이터셋을 넣어 학습하기. 
count_vect.fit(X_train)

#(3) transform() 메서드를 이용해 벡터화하기. 
X_train_cv = count_vect.transform(X_train)
X_test_cv = count_vect.transform(X_test)

In [31]:
#19. Grid search CV로 최적 파라미터 찾기. 
#(1) 모델 설정하기.
lr = LogisticRegression()

#(2) 모델의 하이퍼 파라미터 확인하기. 
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [65]:
#(3) GridSearch를 위한 하이퍼파라미터 값 지정하기. (recycle)
params = {
    'C'     : [0,1,2,3]
}

#(4) GridsearchCV() 메서드를 이용해 훈련 모델 할당하기. 
gscv_lr = GridSearchCV(lr, param_grid=params)

#(5) 모형 학습하기. 
gscv_lr.fit(X_train_cv,y_train)

GridSearchCV(estimator=LogisticRegression(), param_grid={'C': [0, 1, 2, 3]})

In [66]:
#(6) 최적 하이퍼 파라미터 확인하기. 
gscv_lr.best_params_

{'C': 1}

In [67]:
#(7) 최적 파라미터에 대한 평가 점수 확인하기. 
gscv_lr.best_score_

0.8263198309009931

In [68]:
#(8) test 데이터셋으로 모형 예측 및 평가하기. 
accuracy_score(y_test,gscv_lr.predict(X_test_cv))

0.8261659352995203

In [69]:
#20. 실제 데이터로 테스트 해보기. 
#(1) 리뷰 데이터 불러오기. 
review1 = '아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~'
review2 = '메시지와 작위성의 불협화음!!!'

#(2) 텍스트 데이터 전처리하기. 
#①review1 데이터 처리하기. 
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review1)
review1

'아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요'

In [70]:
#②review2 데이터 처리하기. 
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review2)
review2

'메시지와 작위성의 불협화음'

In [71]:
#MEMO. 기존 replace() 메서드로는 작동이 안됨. WHY?

In [72]:
#(3) 형태소 분석후 다시 스트링으로 변경하기. 
#①review1 데이터 처리하기. 
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join([word for word in morphs if not word in stopwords])
review1

'아름답다 음악 아름답다 풍 광 그렇다 못 현실 찡하다'

In [73]:
#②review2 데이터 처리하기. 
morphs = okt.morphs(review2, stem=True)
review2 = ' '.join([word for word in morphs if not word in stopwords])
review2

'메시지 작위 성의 불협화음'

In [74]:
#(4) 벡터화 처리하기. 
review1_cv = count_vect.transform([review1])
review2_cv = count_vect.transform([review2])

#(5) gscv_lr 모델로 긍정, 부정 예측하기.
#①review1 데이터 처리하기. 
review1_pred = gscv_lr.predict(review1_cv)
review1_pred[0]

1

In [75]:
#②review2 데이터 처리하기. 
review2_pred = gscv_lr.predict(review2_cv)
review2_pred[0]

0

In [76]:
#PLUS. for문을 통해 한번에 처리하기. 
#(1) 리뷰 데이터 불러오기. 
reviews = ['아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~',
           '메시지와 작위성의 불협화음!!!']

#(2) 데이터 전처리 및 형태소분석 후 리스트로 변환하기. 
review_list = []
for review in reviews:
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review)
    morphs = okt.morphs(review, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    review_list.append(temp_X)

#(3) 벡터화 처리하기. 
review_cv = count_vect.transform(review_list)

#(4) 모델로 긍정, 부정 예측하기. 
review_pred = gscv_lr.predict(review_cv)
review_pred

array([1, 0], dtype=int64)